1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию,  
которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии   
с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).   


In [114]:
import pymongo
import json
import requests
import pprint
from bs4 import BeautifulSoup as bs
import re
from time import sleep
import pandas as pd
import numpy as np
from pymongo import MongoClient

### Сбор страниц с запросом

In [115]:
def search_vacancy():
    

    """ Функция принимает запрос на вакансию от пользователяи возвращает список стр 
    """
    
    from bs4 import BeautifulSoup as bs
    from requests import get
    # Сборка url
    vacancy_name = input('Введите название вакансии - ')
    url = r'https://hh.ru/search/vacancy?'

    param = {'text': vacancy_name,\
             'salary':'', \
             'ored_clusters':'true', \
             'items_on_page':'20', \
             'enable_snippets':'true', \
             'only_with_salary': 'true', \
             'page':''}
    
    headers = {'User-Agent': 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    
    response = get(url, params=param, headers=headers)
    ful_url = response.url

    if response.ok:
        soup = bs(response.content, 'lxml')
        result_of_requie = soup.select_one('h1.bloko-header-section-3').text.split(' ')# список результатов по запросу
        if result_of_requie[0] =='По':
            return f'По запросу {vacancy_name} ничего не найдено'
        else:
            number_of_vac = int(result_of_requie[0].replace(u'\xa0', u'')) # Замена разделителя \xa в 1м элементе списка (содержит число вакансии)
            number_of_pages = int(round(number_of_vac/20,0)-1) # выесняем чтсло стр (на 1 стр 50 вакансий)
            #list_of_urls = [ful_url + str(i) for i in range(number_of_pages)]
    return [ful_url + str(i) for i in range(number_of_pages)]

list_of_links = search_vacancy()




Введите название вакансии - data science


### Предварительная сборка инфо

In [116]:
def parser_info(list_urls):
    
    """ 
    """
    from bs4 import BeautifulSoup as bs
    import requests
    headers = {'User-Agent': 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    
    
    name =[]
    money =[]
    links = []
    location = []
    vacancy_dict = {}
    for site in list_urls:
        response=requests.get(site, headers=headers)
        sleep(1)
        soup = bs(response.content, 'lxml')
        
        for el in soup.select('a.serp-item__title'):
            name.append(el.text)
        
        for el in soup.select('span.bloko-header-section-3'):
            money.append(el.text)

        for el in soup.find_all('a', {'class': 'serp-item__title'}):
            link = el.get('href')
            links.append(link)      
        for el in soup.find_all('div', {'data-qa': 'vacancy-serp__vacancy-address'}):
            location.append(el.text)
    vacancy_dict['name'] =  name
    vacancy_dict['money'] = money
    vacancy_dict['links'] = links
    vacancy_dict['location'] = location
    
    
    return vacancy_dict

info_json = parser_info(list_of_links)


### Коректировка данных и добавление новых полей

In [117]:
def correction_data(info_json):
    """ 
    Коректирует поле money и разносить его на 3 отдельных столбца  и коректирует вид ссылки  """
   
    min_money = []
    max_money = []
    currency = []
    data_lists = []
    for sentence in info_json['money']:
        sentence = sentence.replace(u"\u202f", u"")
        data_list = re.split(r'\s+|–', sentence)
        data_lists.append(data_list)


    for index, list_of_numbers in enumerate(data_lists):
        if list_of_numbers[0] == 'от':
            min_money.append(None)
            max_money.append(int(list_of_numbers[1]))
            currency.append(list_of_numbers[2])
        elif list_of_numbers[0] == 'до':
            min_money.append(int(list_of_numbers[1]))
            max_money.append('None')
            currency.append(list_of_numbers[2])
        else:
            min_money.append(int(list_of_numbers[0]))
            max_money.append(int(list_of_numbers[3]))
            currency.append(list_of_numbers[4])
            
    info_json['min_money'] = min_money
    info_json['max_money'] = max_money
    info_json['currency'] = currency
    info_json.pop('money')
    df = pd.DataFrame(info_json)
    df = df.replace(['None'], np.NaN)
    df = df.astype({'max_money': np.float64 })
    df_new = df['links'].str.split('?', expand=True)
    df['links'] = df_new[0]
    
    return df
        
df = correction_data(info_json)

### Отображение ЗП больше определеной суммы

In [119]:
def salary_more_than(df):
    
    curency = input('Введите валюту руб., EUR, USD, KZT, сум - ')
    if curency == 'руб.' or curency == 'руб' or curency == 'Руб' or curency == 'РУБ':
        pre_view = df.loc[df['currency']=='руб.']
        try:
            salary = int(input('Введите суму от которой хотите уведить вакансии - '))
            view1 = pre_view.loc[df['min_money']>=salary]
            view2 = pre_view.loc[df['max_money']>=salary] 
            view = view1.merge(view2, how='outer')
            view.drop_duplicates(keep='first',inplace=True)
            return view
        except ValueError:
            return "Введены не целые числа"
        
    elif curency == 'EUR':
        pre_view = df.loc[df['currency']=='EUR']
        try:
            salary = int(input('Введите суму от которой хотите уведить вакансии - '))
            view1 = pre_view.loc[df['min_money']>=salary]
            view2 = pre_view.loc[df['max_money']>=salary] 
            view = view1.merge(view2, how='outer')
            view.drop_duplicates(keep='first',inplace=True)
            return view
        except ValueError:
            return "Введены не целые числа"
        
        
    elif curency == 'USD':
        pre_view = df.loc[df['currency']=='USD']  
        try:
            salary = int(input('Введите суму от которой хотите уведить вакансии - '))
            view1 = pre_view.loc[df['min_money']>=salary]
            view2 = pre_view.loc[df['max_money']>=salary] 
            view = view1.merge(view2, how='outer')
            view.drop_duplicates(keep='first',inplace=True)
            return view
        except ValueError:
            return "Введены не целые числа"
        
        
    elif curency == 'KZT':
        pre_view = df.loc[df['currency']=='KZT']
        try:
            salary = int(input('Введите суму от которой хотите уведить вакансии - '))
            view1 = pre_view.loc[df['min_money']>=salary]
            view2 = pre_view.loc[df['max_money']>=salary] 
            view = view1.merge(view2, how='outer')
            view.drop_duplicates(keep='first',inplace=True)
            return view
        except ValueError:
            return "Введены не целые числа"
        
        
        
    elif curency == 'сум':
        pre_view = df.loc[df['currency']=='сум'] 
        try:
            salary = int(input('Введите суму от которой хотите уведить вакансии - '))
            view1 = pre_view.loc[df['min_money']>=salary]
            view2 = pre_view.loc[df['max_money']>=salary] 
            view = view1.merge(view2, how='outer')
            view.drop_duplicates(keep='first',inplace=True)
            return view
        except ValueError:
            return "Введены не целые числа"
        
        
    else:
        return "Введена не верная валюта"
        
                        

In [120]:
salary_more_than(df)

Введите валюту руб., EUR, USD, KZT, сум - USD
Введите суму от которой хотите уведить вакансии - 1500


,name,links,location,min_money,max_money,currency
0,Математик - программист (math modeling),https://vladimir.hh.ru/vacancy/77282996,Новосибирск,4000.0,10000.0,USD
1,Machine Learning Engineer,https://vladimir.hh.ru/vacancy/76983946,Баку,4000.0,8000.0,USD
2,Data scientist (math modeling),https://vladimir.hh.ru/vacancy/76674451,Москва,4000.0,10000.0,USD
3,"Bioinformatics Developer, New Zealand",https://vladimir.hh.ru/vacancy/76866591,Москва,5000.0,7500.0,USD
4,"Bioinformatics Developer, New Zealand",https://vladimir.hh.ru/vacancy/76866592,Санкт-Петербург,5000.0,7500.0,USD
5,Machine Learning Engineer,https://vladimir.hh.ru/vacancy/76735944,Тбилиси,4000.0,8000.0,USD
6,Quantitative researcher,https://vladimir.hh.ru/vacancy/76567518,Москва,5000.0,7000.0,USD
7,Data Steward Intern,https://vladimir.hh.ru/vacancy/76289753,Абхазия,3800.0,NaN,USD
8,Data Steward Intern,https://vladimir.hh.ru/vacancy/76289755,Цхинвал,3800.0,NaN,USD
9,Data Analyst (американская компания),https://vladimir.hh.ru/vacancy/76437346,Тбилиси,4000.0,7000.0,USD


### Добавление в базу данных

In [121]:
client = MongoClient()

In [35]:
HH_base = client['HH_base']

In [126]:
js = df.to_dict('records')

### Добавление в базу данных

In [127]:
for el in js:
    HH_base.hh_vacancy.insert_one(el)

### Еще 1 парсинг данных по др вакансии

In [129]:
list_of_vacancy_2 = search_vacancy()
js2 = parser_info(list_of_vacancy_2)
df2 = correction_data(js2)
js2 = df2.to_dict('records')

Введите название вакансии - ml


### Выгрузка из базы в датафрейм

In [157]:
list_of_json = []
for el in HH_base.hh_vacancy.find():
    list_of_json.append(el)
df4 = pd.DataFrame(list_of_json)   
df4.drop(columns='_id', inplace=True)
df4

,name,links,location,min_money,max_money,currency
0,Data scientist,https://vladimir.hh.ru/vacancy/74376536,Москва,200000.0,240000.0,руб.
1,Data-engeneer (Middle),https://vladimir.hh.ru/vacancy/76921562,Москва,250000.0,300000.0,руб.
2,Quantitative Analyst,https://vladimir.hh.ru/vacancy/76506452,Кипр,5000.0,8000.0,EUR
3,Senior Data Scientist,https://vladimir.hh.ru/vacancy/73387970,"Москва, Технопарк",350000.0,NaN,руб.
4,Математик - программист (math modeling),https://vladimir.hh.ru/vacancy/77282996,Новосибирск,4000.0,10000.0,USD
...,...,...,...,...,...,...
175,Системный администратор,https://vladimir.hh.ru/vacancy/76323647,Москва,100000.0,NaN,руб.
176,Senior PHP Developer,https://vladimir.hh.ru/vacancy/76276201,Москва,NaN,5500.0,EUR
177,Сhief Technical Officer (CTO),https://vladimir.hh.ru/vacancy/76234573,Москва,NaN,420000.0,руб.
178,Senior system analyst (Brokerage),https://vladimir.hh.ru/vacancy/76145317,Москва,250000.0,400000.0,руб.


In [158]:
df_merged = df4.merge(df2, how='outer')
df_merged.drop_duplicates(keep='first')
df_merged # у нас в df было 180 строк в df2 220 сейчас 357 значит дубли удалились

,name,links,location,min_money,max_money,currency
0,Data scientist,https://vladimir.hh.ru/vacancy/74376536,Москва,200000.0,240000.0,руб.
1,Data-engeneer (Middle),https://vladimir.hh.ru/vacancy/76921562,Москва,250000.0,300000.0,руб.
2,Quantitative Analyst,https://vladimir.hh.ru/vacancy/76506452,Кипр,5000.0,8000.0,EUR
3,Senior Data Scientist,https://vladimir.hh.ru/vacancy/73387970,"Москва, Технопарк",350000.0,NaN,руб.
4,Математик - программист (math modeling),https://vladimir.hh.ru/vacancy/77282996,Новосибирск,4000.0,10000.0,USD
...,...,...,...,...,...,...
352,Middle PHP разработчик,https://vladimir.hh.ru/vacancy/76387418,"Москва, Славянский бульвар",170000.0,200000.0,руб.
353,Account manager,https://vladimir.hh.ru/vacancy/76446254,"Москва, Славянский бульвар",NaN,70000.0,руб.
354,Senior Backend Developer (Go) (to Lisbon),https://vladimir.hh.ru/vacancy/37790906,"Москва, Динамо и еще 2",NaN,380000.0,руб.
355,Middle Python-разработчик (удалённо),https://vladimir.hh.ru/vacancy/76435882,Санкт-Петербург,NaN,150000.0,руб.


### Удаляем базу данных и перезаписываем все
В случае больших данных конечно увеличятся временые затраты на весь процес но пока лучшего решения у меня нет

In [168]:
HH_base.hh_vacancy.drop()

In [169]:
for el in HH_base.hh_vacancy.find():
    print(el)

### Перезагружаем данные

In [161]:
HH_base = client['HH_base']

In [170]:
js4 = df_merged.to_dict('records')

In [171]:
for el in js4:
    HH_base.hh_vacancy.insert_one(el)

In [172]:
len_list =[]
for el in HH_base.hh_vacancy.find():
    len_list.append(el)
len(len_list)
    

357

### Оформим действие выше в функцию

In [174]:
def mongo_upload(df_for_upload):
    
    client = MongoClient()
    HH_base = client['HH_base']
    # чтение базы данных в список
    list_of_json = []
    for el in HH_base.hh_vacancy.find():
        list_of_json.append(el)
    df_from_mongo = pd.DataFrame(list_of_json)   
    df_from_mongo.drop(columns='_id', inplace=True)
    
    # Мерджерим и удаляем дубли
    df_merged = df_from_mongo.merge(df_for_upload, how='outer')
    df_merged.drop_duplicates(keep='first')
    
    json = df_merged.to_dict('records')
    
    # Удаляем и пересоздаем базу
    HH_base.hh_vacancy.drop()
    HH_base = client['HH_base']
    # добавляем данные в базу
    for el in js4:
        HH_base.hh_vacancy.insert_one(el) 

In [175]:
mongo_upload(df2)

In [178]:
len_list = []

for el in HH_base.hh_vacancy.find():
    len_list.append(el)
print(len(len_list))

357
